<a href="https://colab.research.google.com/github/brunalvess1991/TechLead-2025.1-FerramentasDeIA/blob/main/books/aula_03_model_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain: Models, Prompts and Output Parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [ ]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/pedrodiamel/agents-mini-course/blob/course/books/aula_03_model_prompt.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

In [ ]:
import os
import openai
from utils import format_message, show_prompt


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [ ]:
# Set the model variable based on the current date
llm_model = "gpt-4o-mini"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [ ]:
# Ref: https://platform.openai.com/docs/api-reference/chat/create

client = openai.OpenAI()

def get_completion(prompt, model=llm_model):
    messages = [
        {
        "role": "user",
        "content": prompt
        }
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content


In [ ]:
client = openai.OpenAI()

with client.chat.completions.stream(
    model=llm_model,
    messages=[{"role": "user", "content": "Explique brevemente o que é aprendizado de máquina."}],
    temperature=0,
) as stream:
    for event in stream:
        if event.type == "message.delta":
            print(event.delta, end="", flush=True)

In [ ]:
response = get_completion("Explique brevemente o que é aprendizado de máquina.")
print(response)

Aprendizado de máquina é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados. Em vez de serem programados explicitamente para realizar uma tarefa, os sistemas de aprendizado de máquina usam padrões e inferências a partir de dados para melhorar seu desempenho ao longo do tempo. Isso pode incluir tarefas como classificação, regressão, reconhecimento de padrões e tomada de decisões. O aprendizado de máquina é amplamente utilizado em diversas aplicações, como reconhecimento de voz, recomendação de produtos, detecção de fraudes e muito mais.


In [ ]:
styles = ["formal and technical", "casual and friendly", "enthusiastic and persuasive", "concise and to the point", "storytelling and engaging"]
tones = ["confident", "empathetic", "urgent", "optimistic", "serious"]

In [ ]:
prompt_task = """
Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.

<Task>
Redigir um email com base no contexto fornecido.
</Task>

O email precisa ter a seguinte estrutura básica:
<Structure>
1. Saudação
2. Introdução ao problema
3. Apresentação da solução
4. Benefícios da solução
5. Chamada para ação (CTA)
6. Despedida
</Structure>
"""

prompt_context = """
<Context>
Escreva um email para um cliente corporativo com o seguinte contexto:
{customer_email}
O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.
</Context>
"""

prompt_instructions = """
<Instrutions>
Use o estilo e tom especificados para redigir o email.
Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.
Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.
Não invente informações; baseie-se apenas no contexto fornecido.
Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.
</Instrutions>
"""

prompt_references = """
<Example>
Por exemplo, um email formal e técnico com tom confiante pode ser:
Asunto: Apresentação de Solução Avançada de Visão Computacional para Inspeção Industrial
Prezado Sr. Silva,
Gostaria de apresentar nossa avançada solução de visão computacional projetada para otimizar processos de inspeção industrial.
Nossa tecnologia utiliza algoritmos de ponta para garantir precisão e eficiência, reduzindo custos operacionais.
Ficaria honrado em agendar uma demonstração para discutir como nossa solução pode beneficiar sua empresa.
Atenciosamente,
João Pereira
</Example>
"""

prompt = f"""{prompt_task}
{prompt_context}
{prompt_instructions}
{prompt_references}
"""


In [ ]:
show_prompt(prompt)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                                                                                 │
│                                                       

In [ ]:
customer_email = """
Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.
"""

print(customer_email)


Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.



In [ ]:
from datetime import datetime
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

prompt.format(customer_email=customer_email, style=styles[0], tone=tones[0], date=get_today_str())
response = get_completion(prompt)

In [ ]:
print(response)

Assunto: Proposta de Solução para Otimização de Processos

Prezado [Nome do Cliente],

Espero que este email o encontre bem.

Identificamos que sua empresa enfrenta desafios na otimização de processos, o que pode impactar a eficiência e a produtividade. Sabemos que, em um mercado competitivo, é crucial encontrar maneiras de melhorar continuamente.

Para abordar essa questão, gostaríamos de apresentar nossa solução inovadora de automação de processos. Nossa plataforma foi desenvolvida para integrar-se facilmente aos sistemas existentes, proporcionando uma gestão mais eficiente e reduzindo o tempo gasto em tarefas manuais.

Os benefícios dessa solução incluem:
- Aumento significativo na eficiência operacional.
- Redução de erros humanos e retrabalho.
- Melhoria na alocação de recursos, permitindo que sua equipe se concentre em atividades estratégicas.

Estamos confiantes de que nossa solução pode trazer resultados positivos para sua empresa. Gostaríamos de agendar uma demonstração para q

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [42]:
!pip install --upgrade langchain
!pip install langchain_openai

### Model

In [43]:
from langchain_openai import ChatOpenAI

In [44]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    temperature=0.0,
    model=llm_model
    )
chat

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x7d2b54a6f8f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7d2b3af64ec0>, root_client=<openai.OpenAI object at 0x7d2b3af6d130>, root_async_client=<openai.AsyncOpenAI object at 0x7d2b3af66330>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_organization='org-5qZfsmr4Uz6f4Olo3UqM4Fgb', stream_usage=True)

In [45]:
from langchain_core.messages import HumanMessage

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = chat.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [46]:
from utils import format_message, show_prompt

show_prompt(prompt)
format_message([response])

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Assunto: Proposta de Solução para Otimização de Processos                                                       │
│                                                                                                                 │
│ Prezado [Nome do Cliente],                                                                                      │
│                                                                                                                 │
│ Espero que este email o encontre bem. Estou entrando em contato para discutir um desafio que muitas empresas    │
│ enfrentam atualmente: a necessidade de otimizar processos para aumentar a eficiência e reduzir custos.          │
│                                                                                                                 │
│ Para abordar essa questão, nossa equipe desenvolveu uma solução inovadora que integra tecnologia de automação e │
│ análise de dados. Essa ferramenta não apenas simplifica os fluxos de trabalho, mas também proporciona insights  │
│ valiosos que podem impulsionar a tomada de decisões estratégicas.                                               │
│                                                                                                                 │
│ Os benefícios dessa solução são significativos: aumento da produtividade, redução de erros operacionais e,      │
│ consequentemente, uma economia considerável nos custos. Além disso, ao implementar essa tecnologia, sua empresa │
│ se posicionará como líder em inovação no setor.                                                                 │
│                                                                                                                 │
│ Gostaria de convidá-lo para agendar uma demonstração, onde poderemos explorar juntos como essa solução pode ser │
│ adaptada às necessidades específicas da sua empresa. Por favor, me avise sobre sua disponibilidade para que     │
│ possamos marcar um horário conveniente.                                                                         │
│                                                                                                                 │
│ Agradeço pela atenção e fico à disposição para qualquer dúvida.                                                 │
│                                                                                                                 │
│ Atenciosamente,                                                                                                 │
│                                                                                                                 │
│ [Seu Nome]                                                                                                      │
│ [Seu Cargo]                                                                                                     │
│ [Nome da Empresa]                                                                                               │
│ [Telefone]                                                                                                      │
│ [Email]                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Format output

In [47]:
from pydantic import BaseModel, Field

class EmailResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    tips: str = Field(..., description="Additional tips for improving the email.")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")


In [48]:
email_writer_chain = (
    chat
    .with_structured_output(EmailResponse)
)

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = email_writer_chain.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [ ]:
response

EmailResponse(customer_email='Assunto: Proposta de Solução para Aumento de Eficiência Operacional\n\nPrezado Sr. Almeida,\n\nEspero que este email o encontre bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de aumentar a eficiência operacional em um ambiente de negócios cada vez mais competitivo.\n\nPara abordar essa questão, nossa equipe desenvolveu uma solução inovadora que integra tecnologia de automação com análise de dados em tempo real. Essa abordagem não apenas simplifica processos, mas também proporciona insights valiosos que podem ser utilizados para otimizar a tomada de decisões.\n\nOs benefícios dessa solução incluem a redução de custos operacionais, aumento da produtividade e a capacidade de responder rapidamente às mudanças do mercado. Com a implementação dessa tecnologia, sua empresa poderá não apenas se manter competitiva, mas também se destacar em seu setor.\n\nGostaria de convidá-lo para agendar uma demonst

In [ ]:
print("Customer Email:")
print(response.customer_email)
print("\nTips:")
print(response.tips)
print("\nChain of Thought:")
print(response.chain_of_thought)
print("\nScore:")
print(response.score)

Customer Email:
Assunto: Proposta de Solução para Aumento de Eficiência Operacional

Prezado Sr. Almeida,

Espero que este email o encontre bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de aumentar a eficiência operacional em um ambiente de negócios cada vez mais competitivo.

Para abordar essa questão, nossa equipe desenvolveu uma solução inovadora que integra tecnologia de automação com análise de dados em tempo real. Essa abordagem não apenas simplifica processos, mas também proporciona insights valiosos que podem ser utilizados para otimizar a tomada de decisões.

Os benefícios dessa solução incluem a redução de custos operacionais, aumento da produtividade e a capacidade de responder rapidamente às mudanças do mercado. Com a implementação dessa tecnologia, sua empresa poderá não apenas se manter competitiva, mas também se destacar em seu setor.

Gostaria de convidá-lo para agendar uma demonstração, onde poderemos ex

### Exercisio

Para aprimorar os resultados obtidos vamos a criar um sistemas multi-agente para conseguir fazer uma análise do problema (reflexão), escrever o email a partir deste plano (escrita) e finalmente avaliar os resultados obtidos (avaliação).

- Step 1: Crie os três agentes (Agente reflexão, Agente escrita, Agente avaliação)
- Step 2: Crie um pipeline (chain) para gerar os resultados
- Step 3: Avalie e intere sobre os resultados obtidos


In [ ]:
# agent <llm.with(Result).prompt>


In [86]:
#agent_response

class EmailResponse(BaseModel):
    response_email: str = Field(..., description="The full email content generated for the customer.")
    tips: str = Field(..., description="Practical advice on the best time to send or how to follow up.")
    chain_of_thought: str = Field(..., description="The reasoning behind choosing specific words and arguments.")
    score: int = Field(..., description="A score from 1 to 10 evaluating how well the email meets the strategic goals.")

In [91]:
email_response_chain = chat.with_structured_output(EmailResponse)

# O input deve conter o contexto do problema e as preferências de estilo
input_data = {
    "context": "Aumento de 15% nos custos de infraestrutura devido à atualização para servidores com proteção contra ataques DDoS e maior velocidade de cache.",
    "style": "Profissional e Direto",
    "tone": "Parceria Estratégica"
}

prompt_final = f"""
Com base no contexto: {input_data['context']}
Escreva um e-mail seguindo o estilo {input_data['style']} e tom {input_data['tone']}.
Data de hoje: {get_today_str()}
"""

response = email_response_chain.invoke([
    HumanMessage(content=prompt_final)
])

print(f"E-mail: {response.response_email}")
#print(f"Raciocínio: {response.chain_of_thought}")

E-mail: Assunto: Atualização sobre Custos de Infraestrutura

Prezados [Nome do Destinatário],

Espero que esta mensagem os encontre bem.

Gostaria de informar que, em decorrência da recente atualização de nossa infraestrutura, que inclui a implementação de servidores com proteção contra ataques DDoS e um aumento na velocidade de cache, observamos um aumento de 15% nos custos operacionais. Essa atualização é fundamental para garantir a segurança e a eficiência de nossos serviços, proporcionando uma experiência ainda melhor para nossos clientes.

Estamos comprometidos em manter a transparência em nossa parceria e acreditamos que esses investimentos são essenciais para fortalecer nossa posição no mercado e garantir a continuidade de um serviço de alta qualidade. 

Agradecemos pela compreensão e estamos à disposição para discutir como podemos continuar a colaborar de forma eficaz, mesmo diante desses novos desafios.

Atenciosamente,

[Seu Nome]  
[Seu Cargo]  
[Nome da Empresa]  
[Telefone

In [93]:
#agent_reflection
class EmailReflection(BaseModel):
    email_reflection: str = Field(..., description="The final, polished version of the email after reflection.")
    critique: str = Field(..., description="A detailed critique of what was wrong or weak in the initial draft.")
    chain_of_thought: str = Field(..., description="The internal reasoning process: why these specific changes were made.")
    score: int = Field(..., description="A quality score from 1 to 10 for the final version (goal is 9+).")





In [94]:
HumanMessage(content=f"Contexto: {input_data}. Estilo: {styles[0]}.")

# --- PONTE DE INTERLIGAÇÃO ---
# Aqui pegamos o texto que o Agente 1 escreveu para enviar ao Agente de Reflexão
texto_para_revisar = response.response_email

# --- AGENTE 2: REFLEXÃO (Recebe a saída do Agente 1) ---
prompt_reflection = f"""
Você recebeu o seguinte rascunho de e-mail para revisar:
---
{response.response_email}
---
Considere o contexto original: {input_data}
Por favor, realize sua reflexão crítica e entregue a versão final otimizada, destacando que a não execução da atualização pode derrubar o sistema.
"""

# O Agente de Reflexão agora processa o que o Escritor fez
resultado_final = email_reflection_chain.invoke([
    HumanMessage(content=prompt_reflection)
])

# --- RESULTADO FINAL ---
print("E-mail após reflexão:")
print(resultado_final)
print(f"Nota de qualidade: {resultado_final.score}")

E-mail após reflexão:
email_reflection='Assunto: Atualização sobre Custos de Infraestrutura\n\nPrezados [Nome do Destinatário],\n\nEspero que esta mensagem os encontre bem.\n\nGostaria de informar que, em decorrência da recente atualização de nossa infraestrutura, que inclui a implementação de servidores com proteção contra ataques DDoS e um aumento na velocidade de cache, observamos um aumento de 15% nos custos operacionais. Essa atualização é fundamental para garantir a segurança e a eficiência de nossos serviços, proporcionando uma experiência ainda melhor para nossos clientes.\n\nÉ importante ressaltar que a não execução dessa atualização pode comprometer a estabilidade do sistema, aumentando o risco de interrupções e impactando negativamente a qualidade do serviço que oferecemos. Estamos comprometidos em manter a transparência em nossa parceria e acreditamos que esses investimentos são essenciais para fortalecer nossa posição no mercado e garantir a continuidade de um serviço de a

In [95]:
#agent_evaluation
from pydantic import BaseModel, Field

class EvaluationResponse(BaseModel):
    approved: bool = Field(..., description="True if the email is ready to be sent, False if it needs more changes.")
    strengths: list[str] = Field(..., description="List of positive aspects of the email.")
    weaknesses: list[str] = Field(..., description="List of points that could still be improved.")
    final_score: int = Field(..., description="A final strict score from 1 to 10.")
    chain_of_thought: str = Field(..., description="The auditor's reasoning process.")



In [102]:
# 1. Inicializa o Agente de Avaliação
auditor_chain = chat.with_structured_output(EvaluationResponse)

# 2. Definição do Agente
from langchain_core.messages import SystemMessage, HumanMessage

# Definindo a mensagem de sistema
instrucoes_do_agente = "Você é um especialista em logística que fala de forma curta e grossa."

# Passando para a chain
resposta = auditor_chain.invoke([
    SystemMessage(content=instrucoes_do_agente), # Aqui define o comportamento
    HumanMessage(content="Onde está meu pacote?") # Aqui é a pergunta do usuário
])

# 3. Pega a saída do Agente de Reflexão (que você já rodou)
email_para_auditar = resultado_final # Pegando o texto final do reflexão

# 4. Prompt de Interligação
prompt_auditoria = f"""
AUDITE O SEGUINTE E-MAIL:
---
{email_para_auditar}
---
CONTEXTO ORIGINAL: {input_data}
ESTILO SOLICITADO: {styles[0]}
TOM SOLICITADO: {tones[0]}

Realize uma auditoria fria e técnica.
"""

# 4. Execução do Último Agente
veredito_final = auditor_chain.invoke([
    SystemMessage(content="Você é o Auditor Final de Qualidade."),
    HumanMessage(content=prompt_auditoria)
])

# 5. Exibição do Resultado
print(f"--- VEREDITO FINAL ---")
print(f"Aprovado para Envio: {'SIM' if veredito_final.approved else 'NÃO'}")
print(f"Nota Final: {veredito_final.final_score}/10")
print(f"Pontos Fortes: {veredito_final.strengths}")
print(f"Raciocínio do Auditor: {veredito_final.chain_of_thought}")

--- VEREDITO FINAL ---
Aprovado para Envio: SIM
Nota Final: 9/10
Pontos Fortes: ['Clareza na comunicação dos custos e atualizações de infraestrutura.', 'Inclusão de riscos associados à não execução da atualização, o que reforça a urgência da mensagem.', 'Tom profissional e direto, adequado para o contexto.', 'Estrutura bem organizada, facilitando a leitura e compreensão.']
Raciocínio do Auditor: O e-mail apresenta uma boa estrutura e clareza na comunicação, abordando de forma eficaz o aumento de custos e a importância da atualização de infraestrutura. A inclusão dos riscos associados à não execução da atualização é um ponto positivo que reforça a necessidade do investimento. No entanto, a linguagem poderia ser um pouco mais assertiva para transmitir maior confiança e urgência. Além disso, uma chamada à ação mais clara poderia melhorar a interação com o destinatário. No geral, o e-mail está bem elaborado e pronto para ser enviado.


### Extra: Usando ollama LLMs

In [ ]:
#!pip install langchain-community

In [ ]:
from langchain_community.llms import Ollama

# Conecta ao servidor Ollama (local ou remoto)
llm = Ollama(
    model="gemma2:2b",
    base_url="http://ollama:11434"  # troque se estiver remoto
)

# Prompt simples
response = llm.invoke("Olá! Me diga um 'Olá Mundo' bem bonito em português.")

print("Resposta do modelo:")
print(response)

Resposta do modelo:
Olá, mundo! 🌎✨ 

Que a beleza e o encanto da vida fluam por todo o seu caminho. 😉  😊 



**Rascunho Trabalho atv 3**

In [83]:
#rascunho
email_reflection_chain = (
    chat
    .with_structured_output(EmailReflection)
)

reflection_prompt = """
Você é o Revisor de Elite. Sua missão é pegar o e-mail do 'Escritor' e refiná-lo.
Use a técnica de 'Reflexão em Camadas':
1. Camada de Tom: O e-mail deve ser firme, porém acolhedor.
2. Camada de Precisão: Verifique se os dados de infraestrutura e data estão claros.
3. Camada de Concisão: Remova frases que não agregam valor.
"""

response = chat.invoke([HumanMessage(content=reflection_prompt)])
print("DONE")

DONE


In [84]:
#rascunho
email_response_chain = (
    chat
    .with_structured_output(EmailResponse)
)

input_data = {reflection_prompt}

prompt_response = f"""
Com base no contexto: {input_data}
Escreva um e-mail seguindo o estilo {style[0]}, o tom {tone[0]}.
Data de hoje: {get_today_str()}
"""

response = email_response_chain.invoke([HumanMessage(content=prompt_response)])
print("DONE")

NameError: name 'style' is not defined